In [1]:
from simphony_osp.namespaces import city, rdfschema, dcoretbox, simphony
from simphony_osp.namespaces import owl,foaf, emmo, domeo, dcterms, evmpo, eurosci, iof
from simphony_osp.namespaces import http_method

from simphony_osp.tools import semantic2dot, pretty_print
from simphony_osp.tools import search,  export_file, import_file, pretty_print, search
from simphony_osp.tools.search import sparql

from simphony_osp.ontology import RESTRICTION_QUANTIFIER, RESTRICTION_TYPE, OntologyIndividual
from simphony_osp.ontology import COMPOSITION_OPERATOR, Composition, Restriction

from simphony_osp.ontology.relationship import OntologyRelationship
from simphony_osp.ontology.annotation import OntologyAnnotation

from simphony_osp.session import Session, core_session
import json, rdflib
from rdflib import Graph, URIRef
from rdflib.plugins.parsers.jsonld import to_rdf as json_to_rdf

from json2html import *
from IPython.core.display import display, HTML
import pandas as pd


/var/folders/jd/j0s9pmrj0kg0ww159sk08ls40000gp/T/ipykernel_48953/174612537.py:21: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# CUDS DOME 4.0
Create a DomeDataSet and connect it with all elements from the DOME 4.0 Data Set Ontology.

In [2]:
# Load the DOME 4.0 A-BOX
SIGRA_ROOT='/Users/adham/dev/new/dome/Ontology-matters/'

In [3]:
core_session.clear()

In [4]:
import_file(SIGRA_ROOT+'dome4.0_core_abox.ttl', all_triples=True)

{<OntologyIndividual: AiiDA export file https://dome40.eu/semantics/dome4.0_core#AIIDA_FORMAT>,
 <OntologyIndividual: CAS Number https://dome40.eu/semantics/dome4.0_core#CAS_NUMBER>,
 <OntologyIndividual: CHEMEO https://dome40.eu/semantics/scenario/platforms#CHEMEO>,
 <OntologyIndividual: CONSUMER https://dome40.eu/semantics/dome4.0_core#CONSUMER>,
 <OntologyIndividual: CRYSTALLOGRAPHY https://dome40.eu/semantics/scenario/platforms#CRYSTALLOGRAPHY>,
 <OntologyIndividual: Cartography https://dome40.eu/semantics/dome4.0_core#CARTOGRAPHY>,
 <OntologyIndividual: Comma-separated Values (CSV) https://dome40.eu/semantics/dome4.0_core#CSV>,
 <OntologyIndividual: Compare https://dome40.eu/semantics/dome4.0_core#COMPARE>,
 <OntologyIndividual: Crystallographic Information Framework (CIF) - Model and format https://dome40.eu/semantics/dome4.0_core#CIF>,
 <OntologyIndividual: DATA_ON_DEMAND_PROVIDER https://dome40.eu/semantics/dome4.0_core#DATA_ON_DEMAND_PROVIDER>,
 <OntologyIndividual: DATA_PROVI

## Demonstrate a CUDS DOME 4.0 DataSet

- First lets create an instance (entity in the A-Box of DOME 4.0) of a DomeDataSet

In [5]:
domeo

AttributeError: 'No entity with label or suffix DomeDa was found in namespace domeo (https://dome40.eu/semantics/domeo#).'

In [ ]:
pretty_print(data_set)

In [ ]:
semantic2dot(data_set)

- Let us attach to it all the parts according to D3.1  and D3.2

In [ ]:
title=dcterms.Title.iri
title
print(title)

In [ ]:
type(title)

In [ ]:
# this is the simphony way
res = sparql(
    f"""SELECT ?s WHERE {{
        ?s a owl:Class ;
            rdfs:label ?label .
            FILTER (CONTAINS(STR(?s), "k") || CONTAINS(STR(?label), "k"))
    }}
    """, ontology=True)

In [ ]:
def search_ontology(some_string):
    # this is the simphony way
    res = sparql(
        f"""SELECT ?s WHERE {{
            ?s a owl:Class ;
                rdfs:label ?label .
                FILTER (CONTAINS(STR(?s), "{some_string}") || CONTAINS(STR(?label), "{some_string}"))
        }}
        """, ontology=True)
    return json.loads(res.serialize(format="json").decode("utf-8"))

In [ ]:
print(res.serialize(format="json"))

In [ ]:
for i in res:
    print(i)

In [ ]:
res=search_ontology("k")

In [ ]:
res


In [ ]:
res["results"]['bindings']

In [ ]:
def res2list(R, as_json=False):
    results_list = []

    for row in R:
        # Using a dictionary comprehension to get the desired structure
        row_dict = {key: str(value) for key, value in zip(['s', 'p', 'o'], row)}
        results_list.append(row_dict)

    if as_json:
        return json.dumps(results_list, indent=4)
    
    return results_list

In [ ]:
res2list(res)

In [ ]:
def search_ontology(some_string):
    # this is the simphony way
    res = sparql(
        f"""SELECT ?s WHERE {{
            ?s a owl:Class ;
                rdfs:label ?label ;
                skos:preLabel ?prefLabel .
                FILTER (CONTAINS(STR(?s), "{some_string}") || CONTAINS(STR(?label), "{some_string}") || CONTAINS(STR(?prefLabel), "{some_string}") )
        }}
        """, ontology=True)
    return res2list(res)

In [ ]:
res=search_ontology("t")

In [ ]:
res

In [ ]:
def search_ontology(some_string):
    # this is the simphony way
    res = sparql(
        f"""SELECT ?s ?label ?prefLabel WHERE {{
            ?s a owl:Class .
            OPTIONAL {{ ?s rdfs:label ?label . }}
            OPTIONAL {{ ?s skos:prefLabel ?prefLabel . }}
            FILTER (
                CONTAINS(STR(?s), "{some_string}") || 
                (BOUND(?label) && CONTAINS(STR(?label), "{some_string}")) || 
                (BOUND(?prefLabel) && CONTAINS(STR(?prefLabel), "{some_string}"))
            )
        }}
        """, ontology=True)
    return res2list(res)

In [ ]:
res=search_ontology("title")

In [ ]:
res
    

In [ ]:
def search_annotation(some_string):
    # this is the simphony way
    res = sparql(
        f"""SELECT ?s ?label ?prefLabel WHERE {{
            ?s a owl:Annotation .
            OPTIONAL {{ ?s rdfs:label ?label . }}
            OPTIONAL {{ ?s skos:prefLabel ?prefLabel . }}
            FILTER (
                CONTAINS(STR(?s), "{some_string}") || 
                (BOUND(?label) && CONTAINS(STR(?label), "{some_string}")) || 
                (BOUND(?prefLabel) && CONTAINS(STR(?prefLabel), "{some_string}"))
            )
        }}
        """, ontology=True)
    return res2list(res)

In [ ]:
res=search_ontology("title")


In [ ]:
res

In [ ]:
for i in data_set.annotations_iter():
    i

In [ ]:
t=dcterms.title


In [ ]:
t

In [ ]:
#annotations do not work and are evit semantically! 
 data_set.connect(t, "hello")

In [ ]:
core_session.graph

In [ ]:
for s, p, o in core_session.graph:
    print(s, p, o)

In [ ]:
core_session.graph.serialize(format="turtle", destination="test.turtle")

In [ ]:

pwd


In [ ]:
import graphviz

In [ ]:
graphviz.Source.from_file("/Users/adham/dev/new/dome/Ontology-matters/domeo/DomeDataSet.dot")